<a href="https://colab.research.google.com/github/brunoricciotti/ME1/blob/main/TPL3_Calibracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import statistics as st
import numpy as np
import math as mt

mA = 0.001
uA = 0.000001

res = [1e-6,1e-5,1e-4]

def incertidumbre_A(X) -> float:
  return st.stdev(X)/mt.sqrt(len(X))

#Esta funcion es para error en cuentas. Si nos dan el error
#fijo, usamos la formula, adaptando la manera en la que le enviamos los argumentos
def inecrtidumbre_B(er_p, nd, Ci, Xm) -> float:
  return Xm*((er_p/100)+(nd/Ci))/mt.sqrt(3)

def incertidumbre_total_A_B(uA, uB) -> float:
  return mt.sqrt( uA**2 + uB**2 )

# Esta funcion retorna la cantidad de cuentas de un numero 'num' para una cantidad de digitos 'digits'
def cuentas(num, digits) -> int:
  dec = len(str(int(num)))
  x = digits - dec
  return int(num*(10**x))

def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return mt.trunc(number)

    factor = 10.0 ** decimals
    return mt.trunc(number * factor) / factor

#**TENSIONES**

## instrumentacion

*Tester usado: MAS830L*

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO_1/img/error_MAS830L.png?raw=true:, width=100" alt="My Image" width=700>

https://mastech-group.com/na/en/MAS830L

Patron HP 3468b

https://xdevs.com/doc/HP_Agilent_Keysight/HP%203468A,%20B%20Operating.pdf

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO_1/img/error_HP3468B_voltimetro.png?raw=true:, width=100" alt="My Image" width=500>

##valores medios

Las mediciones se realizaron para 4 valores distintos de la fuente, con dos escalas distintas del tester en cada valor.


In [51]:
from typing import List
V_ver = [
         [[0.10352, 0.10351, 0.1035, 0.10352, 0.10355], [0.10206, 0.10259, 0.10258, 0.10257, 0.10257]],
         [[1.03808, 1.03709, 1.03634, 1.03612, 1.03575], [1.02965, 1.02959, 1.02963, 1.02965, 1.02967]],
         [[9.0600, 9.0585, 9.0563, 9.0552, 9.0531, 9.0521, 9.0515, 9.0538, 9.0521, 9.0503], [9.0238, 9.0237, 9.0236, 9.0235, 9.0233, 9.0230, 9.0229, 9.0231, 9.0233, 9.0231]],
         [[24.0052, 24.0043, 24.0035,24.0031, 24.0030], [24.9868, 24.9867, 24.9866, 24.9868, 24.9867]]
        ]

V_med = [
         [[0.1039, 0.1039, 0.1039, 0.1038, 0.1038], [0.102, 0.102, 0.102, 0.102, 0.102]],
         [[1.039, 1.039, 1.038, 1.038, 1.037], [1.03, 1.03, 1.03, 1.03, 1.02]],
         [[9.08, 9.08, 9.08, 9.08, 9.07, 9.07, 9.07, 9.06, 9.06, 9.06], [9.09, 9.09, 9.09, 9.09, 9.09, 9.09, 9.09, 9.09, 9.09, 9.09]],
         [[24, 24, 24, 24, 24], [24, 24, 24, 24, 24]]         
]
digits_ver = 6 # cantidad de digitos del patron
digits_med = 4 # cantidad de digitos del tester propio

V_ver_m=list(list())
V_med_m=list(list())
ecal_m=list(list())


for i in range(len(V_ver)):
  V_ver_m.append(list())
  V_med_m.append(list())
  ecal_m.append(list())
  for j in range(len(V_ver[0])):
    V_ver_m[i].append(float())
    V_med_m[i].append(float())
    ecal_m[i].append(float())

    V_ver_m[i][j] = st.mean(V_ver[i][j])
    V_med_m[i][j] = st.mean(V_med[i][j])
    ecal_m[i][j] = V_med_m[i][j] - V_ver_m[i][j]

print(V_med[0][0])

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{V_med_m[0][0]}
    verdadero:{V_ver_m[0][0]}
    ecal_m:{ecal_m[0][0]}
  escala 2V
    medido:{V_med_m[0][1]}
    verdadero:{V_ver_m[0][1]}
    ecal_m:{ecal_m[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{V_med_m[1][0]}
    verdadero:{V_ver_m[1][0]}
    ecal_m:{ecal_m[1][0]}
  escala 20V
    medido:{V_med_m[1][1]}
    verdadero:{V_ver_m[1][1]}
    ecal_m:{ecal_m[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{V_med_m[2][0]}
    verdadero:{V_ver_m[2][0]}
    ecal_m:{ecal_m[2][0]}
  escala 200V
    medido:{V_med_m[2][1]}
    verdadero:{V_ver_m[2][1]}
    ecal_m:{ecal_m[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{V_med_m[3][0]}
    verdadero:{V_ver_m[3][0]}
    ecal_m:{ecal_m[3][0]}
  escala 600V
    medido:{V_med_m[3][1]}
    verdadero:{V_ver_m[3][1]}
    ecal_m:{ecal_m[3][1]}
""")
#for i in range()
#V_ver_m[0][0] = st.mean(V_ver)
#V_med_m = st.mean(V_med)
#ecal_m = V_med_m - V_ver_m
#print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')


[0.1039, 0.1039, 0.1039, 0.1038, 0.1038]

Tension de fuente 0.1V
  escala 200mV
    medido:0.10386000000000001
    verdadero:0.10352
    ecal_m:0.00034000000000000696
  escala 2V
    medido:0.102
    verdadero:0.102474
    ecal_m:-0.0004740000000000022

Tension de fuente 1V
  escala 2V
    medido:1.0382
    verdadero:1.036676
    ecal_m:0.0015240000000000808
  escala 20V
    medido:1.028
    verdadero:1.029638
    ecal_m:-0.0016380000000000283

Tension de fuente 9V
  escala 20V
    medido:9.071
    verdadero:9.05429
    ecal_m:0.01670999999999978
  escala 200V
    medido:9.09
    verdadero:9.02333
    ecal_m:0.06667000000000023

Tension de fuente 24V
  escala 200V
    medido:24
    verdadero:24.00382
    ecal_m:-0.0038200000000010448
  escala 600V
    medido:24
    verdadero:24.98672
    ecal_m:-0.9867199999999983



##Incertidumbres de tipo A

In [52]:
uA_V_ver=list(list())
uA_V_med=list(list())

for i in range(len(V_ver)):
  uA_V_ver.append(list())
  uA_V_med.append(list())
  for j in range(len(V_ver[0])):
    uA_V_ver[i].append(float())
    uA_V_med[i].append(float())

    uA_V_ver[i][j] = incertidumbre_A(V_ver[i][j])
    uA_V_med[i][j] = incertidumbre_A(V_med[i][j])

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{uA_V_med[0][0]}
    verdadero:{uA_V_ver[0][0]}
  escala 2V
    medido:{uA_V_med[0][1]}
    verdadero:{uA_V_ver[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{uA_V_med[1][0]}
    verdadero:{uA_V_ver[1][0]}
  escala 20V
    medido:{uA_V_med[1][1]}
    verdadero:{uA_V_ver[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{uA_V_med[2][0]}
    verdadero:{uA_V_ver[2][0]}
  escala 200V
    medido:{uA_V_med[2][1]}
    verdadero:{uA_V_ver[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{uA_V_med[3][0]}
    verdadero:{uA_V_ver[3][0]}
  escala 600V
    medido:{uA_V_med[3][1]}
    verdadero:{uA_V_ver[3][1]}
""")



Tension de fuente 0.1V
  escala 200mV
    medido:2.449489742783248e-05
    verdadero:8.366600265341658e-06
  escala 2V
    medido:0.0
    verdadero:0.00010356640381899931

Tension de fuente 1V
  escala 2V
    medido:0.0003741657386773885
    verdadero:0.0004136496101774967
  escala 20V
    medido:0.002000000000000002
    verdadero:1.356465996625428e-05

Tension de fuente 9V
  escala 20V
    medido:0.0027688746209726324
    verdadero:0.0010008274354531077
  escala 200V
    medido:0.0
    verdadero:9.780933834082629e-05

Tension de fuente 24V
  escala 200V
    medido:0.0
    verdadero:0.0004140048308895222
  escala 600V
    medido:0.0
    verdadero:3.741657386765221e-05



##Incertidumbres de tipo B



In [53]:
#valores sacados del datasheet del tester:
nd_ver = 2
er_p_ver = 0.018

nd_med = 3
er_p_med = 0.5

Ci_V_ver=list(list())
Ci_V_med=list(list())
uB_V_ver=list(list())
uB_V_med=list(list())

for i in range(len(V_ver)):
  uB_V_ver.append(list())
  uB_V_med.append(list())
  Ci_V_ver.append(list())
  Ci_V_med.append(list())
  for j in range(len(V_ver[0])):
    uB_V_ver[i].append(float())
    uB_V_med[i].append(float())
    Ci_V_ver[i].append(list())
    Ci_V_med[i].append(list())

    Ci_V_ver[i][j] = cuentas(V_ver_m[i][j], digits_ver)
    uB_V_ver[i][j] = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver[i][j], V_ver_m[i][j])

    Ci_V_med[i][j] = cuentas(V_med_m[i][j], digits_med)
    # uB_V_med[i][j] = inecrtidumbre_B(er_p_med, nd_med, Ci_V_med[i][j], V_med_m[i][j])
uB_V_med[0][0] = 0.0001/mt.sqrt(3)
uB_V_med[0][1] = 0.001/mt.sqrt(3)
uB_V_med[1][0] = 0.001/mt.sqrt(3)
uB_V_med[1][1] = 0.01/mt.sqrt(3)
uB_V_med[2][0] = 0.01/mt.sqrt(3)
uB_V_med[2][1] = 0.1/mt.sqrt(3)
uB_V_med[3][0] = 0.1/mt.sqrt(3)
uB_V_med[3][1] = 1/mt.sqrt(3)


# Para las mediciones con la fuente en 24V, el patron se puso en escala 30V y podemos
# ver en las dos imagenes al principio que para esta escala cambian estos parametros 
uB_V_ver[3][0] = inecrtidumbre_B(er_p = 0.02, nd = 3, Ci = Ci_V_ver[3][0], Xm = V_ver_m[3][0])
uB_V_ver[3][1] = inecrtidumbre_B(er_p = 0.02, nd = 3, Ci = Ci_V_ver[3][1], Xm = V_ver_m[3][1])

# para la escala de 600V cambian los parametros del tester
uB_V_med[3][1] = inecrtidumbre_B(er_p = 0.8, nd = 5, Ci = Ci_V_med[i][j], Xm = V_med_m[i][j])

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{uB_V_med[0][0]}
    verdadero:{uB_V_ver[0][0]}
  escala 2V
    medido:{uB_V_med[0][1]}
    verdadero:{uB_V_ver[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{uB_V_med[1][0]}
    verdadero:{uB_V_ver[1][0]}
  escala 20V
    medido:{uB_V_med[1][1]}
    verdadero:{uB_V_ver[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{uB_V_med[2][0]}
    verdadero:{uB_V_ver[2][0]}
  escala 200V
    medido:{uB_V_med[2][1]}
    verdadero:{uB_V_ver[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{uB_V_med[3][0]}
    verdadero:{uB_V_ver[3][0]}
  escala 600V
    medido:{uB_V_med[3][1]}
    verdadero:{uB_V_ver[3][1]}
""")


Tension de fuente 0.1V
  escala 200mV
    medido:5.7735026918962585e-05
    verdadero:2.2305119359764325e-05
  escala 2V
    medido:0.0005773502691896258
    verdadero:2.2196866597851434e-05

Tension de fuente 1V
  escala 2V
    medido:0.0005773502691896258
    verdadero:0.00011928160239435644
  escala 20V
    medido:0.005773502691896258
    verdadero:0.00011855021486571614

Tension de fuente 9V
  escala 20V
    medido:0.005773502691896258
    verdadero:0.000952496423771561
  escala 200V
    medido:0.05773502691896258
    verdadero:0.000949278966191421

Tension de fuente 24V
  escala 200V
    medido:0.05773502691896258
    verdadero:0.0029449276127874705
  escala 600V
    medido:0.13971876514388945
    verdadero:0.0030584231230278694



##Incertidumbre total de cada magnitud



In [54]:
uc_V_ver=list(list())
uc_V_med=list(list())

for i in range(len(V_ver)):
  uc_V_ver.append(list())
  uc_V_med.append(list())
  for j in range(len(V_ver[0])):
    uc_V_ver[i].append(float())
    uc_V_med[i].append(float())

    uc_V_ver[i][j] = incertidumbre_total_A_B(uA_V_ver[i][j], uB_V_ver[i][j])
    uc_V_med[i][j] = incertidumbre_total_A_B(uA_V_med[i][j], uB_V_med[i][j])

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{uc_V_med[0][0]}
    verdadero:{uc_V_ver[0][0]}
  escala 2V
    medido:{uc_V_med[0][1]}
    verdadero:{uc_V_ver[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{uc_V_med[1][0]}
    verdadero:{uc_V_ver[1][0]}
  escala 20V
    medido:{uc_V_med[1][1]}
    verdadero:{uc_V_ver[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{uc_V_med[2][0]}
    verdadero:{uc_V_ver[2][0]}
  escala 200V
    medido:{uc_V_med[2][1]}
    verdadero:{uc_V_ver[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{uc_V_med[3][0]}
    verdadero:{uc_V_ver[3][0]}
  escala 600V
    medido:{uc_V_med[3][1]}
    verdadero:{uc_V_ver[3][1]}
""")


Tension de fuente 0.1V
  escala 200mV
    medido:6.271629240742287e-05
    verdadero:2.382264363275723e-05
  escala 2V
    medido:0.0005773502691896258
    verdadero:0.00010591836897707048

Tension de fuente 1V
  escala 2V
    medido:0.00068799224801834
    verdadero:0.0004305044722993715
  escala 20V
    medido:0.006110100926607787
    verdadero:0.00011932373378631582

Tension de fuente 9V
  escala 20V
    medido:0.006403124237432823
    verdadero:0.001381631279630444
  escala 200V
    medido:0.05773502691896258
    verdadero:0.0009543045752379705

Tension de fuente 24V
  escala 200V
    medido:0.05773502691896258
    verdadero:0.0029738861182900017
  escala 600V
    medido:0.13971876514388945
    verdadero:0.003058651990578781



##Incertidumbre combinada - funcion error de calculo

In [55]:
uc_ecal=list(list())

for i in range(len(V_ver)):
  uc_ecal.append(list())
  for j in range(len(V_ver[0])):
    uc_ecal[i].append(float())

    uc_ecal[i][j] = mt.sqrt((uc_V_ver[i][j])**2 + (uc_V_med[i][j])**2)

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{uc_ecal[0][0]}
  escala 2V
    medido:{uc_ecal[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{uc_ecal[1][0]}
  escala 20V
    medido:{uc_ecal[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{uc_ecal[2][0]}
  escala 200V
    medido:{uc_ecal[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{uc_ecal[3][0]}
  escala 600V
    medido:{uc_ecal[3][1]}
""")



Tension de fuente 0.1V
  escala 200mV
    medido:6.708838709483719e-05
  escala 2V
    medido:0.0005869855485615436

Tension de fuente 1V
  escala 2V
    medido:0.0008115832883956454
  escala 20V
    medido:0.006111265947966759

Tension de fuente 9V
  escala 20V
    medido:0.0065504889125051514
  escala 200V
    medido:0.05774291324964176

Tension de fuente 24V
  escala 200V
    medido:0.05781156745823358
  escala 600V
    medido:0.13975224035890377



##Factor de expansion

Al tener como minimo 5 mediciones de cada magnitud, aproximamos y tomamos un k95 = 2

In [56]:
k95 = 2 # aproximamos un k=2 ya que en todos los casos se hicieron varias mediciones

##Incertidumbre expandida

In [57]:
Uc_ecal=list(list())

for i in range(len(V_ver)):
  Uc_ecal.append(list())
  for j in range(len(V_ver[0])):
    Uc_ecal[i].append(float())

    Uc_ecal[i][j] = (uc_ecal[i][j])*k95

print(f"""
Tension de fuente 0.1V
  escala 200mV
    medido:{Uc_ecal[0][0]}
  escala 2V
    medido:{Uc_ecal[0][1]}

Tension de fuente 1V
  escala 2V
    medido:{Uc_ecal[1][0]}
  escala 20V
    medido:{Uc_ecal[1][1]}

Tension de fuente 9V
  escala 20V
    medido:{Uc_ecal[2][0]}
  escala 200V
    medido:{Uc_ecal[2][1]}

Tension de fuente 24V
  escala 200V
    medido:{Uc_ecal[3][0]}
  escala 600V
    medido:{Uc_ecal[3][1]}
""")



Tension de fuente 0.1V
  escala 200mV
    medido:0.00013417677418967437
  escala 2V
    medido:0.0011739710971230871

Tension de fuente 1V
  escala 2V
    medido:0.0016231665767912908
  escala 20V
    medido:0.012222531895933518

Tension de fuente 9V
  escala 20V
    medido:0.013100977825010303
  escala 200V
    medido:0.11548582649928352

Tension de fuente 24V
  escala 200V
    medido:0.11562313491646715
  escala 600V
    medido:0.27950448071780754



## verificacion calibracion

PAra que la calibracion sea aceptabe, |ecal_m|+Uc_ecal <= deltaV

In [58]:
delta_V=list(list())
suma=list(list())
verificacion=list(list())

for i in range(len(V_ver)):
  delta_V.append(list())
  suma.append(list())
  verificacion.append(list())
  for j in range(len(V_ver[0])):
    delta_V[i].append(float())
    suma[i].append(float())
    verificacion[i].append(str())

    delta_V[i][j] = (V_med_m[i][j]) * ( (er_p_med/100)+(nd_med/Ci_V_med[i][j]) )
    suma[i][j] = abs(ecal_m[i][j])+ Uc_ecal[i][j]
    print(f'suma: {suma[i][j]}  delta: {delta_V[i][j]}')
    if suma[i][j] <= delta_V[i][j]: verificacion[i][j] = 'SI Verifica calibracion'
    else: verificacion[i][j] = 'NO Verifica calibracion'


print(f"""
Tension de fuente 0.1V
  escala 200mV: {verificacion[0][0]}
  escala 2V: {verificacion[0][1]}

Tension de fuente 1V
  escala 2V: {verificacion[1][0]}
  escala 20V: {verificacion[1][1]}

Tension de fuente 9V
  escala 20V: {verificacion[2][0]}
  escala 200V: {verificacion[2][1]}

Tension de fuente 24V
  escala 200V: {verificacion[3][0]}
  escala 600V: {verificacion[3][1]}
""")



suma: 0.0004741767741896813  delta: 0.0035443485436893207
suma: 0.0016479710971230893  delta: 0.0035099999999999997
suma: 0.0031471665767913715  delta: 0.00819157803468208
suma: 0.013860531895933547  delta: 0.008140000000000001
suma: 0.029810977825010083  delta: 0.048355
suma: 0.18215582649928375  delta: 0.04845
suma: 0.1194431349164682  delta: 0.15000000000000002
suma: 1.2662244807178058  delta: 0.15000000000000002

Tension de fuente 0.1V
  escala 200mV: SI Verifica calibracion
  escala 2V: SI Verifica calibracion

Tension de fuente 1V
  escala 2V: SI Verifica calibracion
  escala 20V: NO Verifica calibracion

Tension de fuente 9V
  escala 20V: SI Verifica calibracion
  escala 200V: NO Verifica calibracion

Tension de fuente 24V
  escala 200V: SI Verifica calibracion
  escala 600V: NO Verifica calibracion



#**CORRIENTE**

## instrumentacion



*Tester usado: DT380B*

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO_1/img/error_DT380B.png?raw=true:, width=100" alt="My Image" width=500>

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO_1/img/error_MAS830L.png?raw=true:, width=100" alt="My Image" width=700>

https://www.viaindustrial.com/manuales_pdf/multimetro-digital-economico-basico-96830-dt830-b-ebchq-manual-espanol.pdf

Patron HP 3468b

https://xdevs.com/doc/HP_Agilent_Keysight/HP%203468A,%20B%20Operating.pdf

<img src="https://github.com/brunoricciotti/ME1/blob/main/LABORATORIO/LABORATORIO_1/img/error_HP3468B.png?raw=true:, width=100" alt="My Image" width=500>

## Procedimiento específico de la medición:
Utilizamos una fuente generadora de corriente, a la cual se le dieron distintos valores de corriente para vertificar. En nuestro caso generaron corrientes de 100uA, 1.5mA y 6.75mA.

Cada una de estas corrientes fue medida usando 2 amperimetros en serie. Uno era el amperimetro patron y el otro era el generico. A su vez por cada valor de corriente se usaron 2 escalas distintas en el amperimetro generico, y por cada escala se tomaron 5 valores de cada amperimetro.

A partir de las valores obtenidos se calculan las incertidumbres, y se determina si se verifica la calibracion.

##**Escala: 2000uA**
##**Fuente: 100uA**

In [19]:
V_ver = [0.00010,0.00010,0.00010,0.00009,0.00009]
V_med = [114,115,115,114,115]#uA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*uA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m*uA}\necal_m: {ecal_m}')

V_ver_m: 9.6e-05
V_med_m: 0.0001146
ecal_m: 1.859999999999999e-05


###Incertidumbres de tipo A

In [20]:
V_med_uA = list(map(lambda x: x*uA, V_med))#paso de uA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_uA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831775e-06
uA_V_med: 2.449489742783171e-07


###Incertidumbres de tipo B


Al calcular las cuentas ci para ambas mediciones, vemos que en **este caso en particular** todas las medidiones obtenidas con nuestro tester son de 3 digitos no decimales. Por eso no es necesario multiplicar por ningun numero para obtener las ci. Para el valor medio patron, como tiene 5 decimales lo multiplicamos por 100000,y al hacer int() los numeros decimamles se truncan. Asi obtenemos las ci.

**(el mismo proceso se repetira para las otras mediciones)**

In [21]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1) #son 3 digitos, y V_med_m esta en uA sin decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[0]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.70446407920678e-05
uB_V_med: 2.886751345948129e-07


###Incertidumbre total de cada magnitud


In [22]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

Incertidumbre total
uc_V_ver: 3.70446407920678e-05
uc_V_med: 3.785938897200178e-07


###Incertidumbre combinada - funcion error de calculo

In [23]:
uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

uc_ecal: 3.704657534437788e-05


###Incertidumbre expandida

In [24]:
k95 = 2

Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

Uc_ecal: 7.409315068875575e-05


### verificacion calibracion

Para que la calibracion sea aceptabe, $|e_{cal_m}|+Uc_{ecal} <= \Delta V$

In [25]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*uA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.409315068875575e-05
delta_V: 3.1565263157894737e-06
|ecal_m|+Uc_ecal = 7.409316928875575e-05
NO verifica calibracion


##**Escala: 20mA**
##**Fuente: 100uA**

In [26]:
V_ver = [0.00010,0.00010,0.00010,0.00009,0.00009]
V_med = [0.12,0.12,0.11,0.11,0.12]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 9.6e-05
V_med_m: 0.11599999999999999
ecal_m: 1.9999999999999998e-05


###Incertidumbres de tipo A

In [27]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831775e-06
uA_V_med: 2.4494897427831775e-06


###Incertidumbres de tipo B

In [28]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*100) #son 3 digitos, y V_med_m esta con 2 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.70446407920678e-05
uB_V_med: 2.8867513459481293e-06


###Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [29]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.70446407920678e-05
uc_V_med: 3.7859388972001827e-06
uc_ecal: 3.723759853624649e-05


###verificacion calibracion

In [30]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.447519707249298e-05
delta_V: 2.2250909090909094e-05
|ecal_m|+Uc_ecal = 7.447521707249298e-05
NO verifica calibracion


##**Escala: 20mA**
##**Fuente: 1,5 mA**

In [31]:
V_ver = [0.00149,0.00149,0.00148,0.00148,0.00149]
V_med = [1.50,1.51,1.51,1.50,1.51]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.001486
V_med_m: 1.506
ecal_m: 2.0000000000000052e-05


###Incertidumbres de tipo A

In [32]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831843e-06
uA_V_med: 2.4494897427831843e-06


###Incertidumbres de tipo B

In [33]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1000) #son 4 digitos, y V_med_m esta con 3 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.623995495336942e-05
uB_V_med: 2.8867513459481293e-06


###Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [34]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.623995495336942e-05
uc_V_med: 3.785938897200187e-06
uc_ecal: 3.6437174264143734e-05


###verificacion calibracion

In [35]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.287434852828747e-05
delta_V: 1.706e-05
|ecal_m|+Uc_ecal = 7.287436852828747e-05
NO verifica calibracion


##**Escala: 2000uA**
##**Fuente: 1,5 mA**

In [36]:
V_ver = [0.00144,0.00144,0.00143,0.00143,0.00144]
V_med = [1462,1461,1462,1462,1461]#uA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*uA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.001436
V_med_m: 1461.6
ecal_m: 2.5599999999999755e-05


###Incertidumbres de tipo A

In [37]:
V_med_uA = list(map(lambda x: x*uA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_uA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427831843e-06
uA_V_med: 2.449489742783503e-07


###Incertidumbres de tipo B

In [38]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*1) #son 4 digitos, y V_med_m esta en uA sin decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[0]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 3.6195790549438136e-05
uB_V_med: 2.886751345948129e-07


###Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [39]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 3.6195790549438136e-05
uc_V_med: 3.785938897200393e-07
uc_ecal: 3.6197770467697704e-05


###verificacion calibracion

In [40]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*uA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*uA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 7.239554093539541e-05
delta_V: 1.661682135523614e-05
|ecal_m|+Uc_ecal = 7.23955665353954e-05
NO verifica calibracion


##**Escala: 20mA**
##**Fuente: 6,75mA**

In [41]:
V_ver = [0.00674,0.00674,0.00673,0.00673,0.00674]
V_med = [6.76,6.76,6.77,6.77,6.76]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.006736
V_med_m: 6.763999999999999
ecal_m: 2.799999999999938e-05


###Incertidumbres de tipo A

In [42]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere
print(V_med_mA)
print(V_ver)
uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

[0.0067599999999999995, 0.0067599999999999995, 0.00677, 0.00677, 0.0067599999999999995]
[0.00674, 0.00674, 0.00673, 0.00673, 0.00674]
Incertidumbre tipo A
uA_V_ver: 2.4494897427832906e-06
uA_V_med: 2.4494897427832906e-06


###Incertidumbres de tipo B

In [43]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 2
er_p_med = 1
Ci_V_med = int(V_med_m*100) #son 3 digitos, y V_med_m esta con 2 decimales

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[1]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 4.128325307500771e-05
uB_V_med: 2.8867513459481293e-06


###Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [44]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 4.128325307500771e-05
uc_V_med: 3.7859388972002555e-06
uc_ecal: 4.145648704109488e-05


###verificacion calibracion

In [45]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*mA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m'+'Verifica calibracion')
else: print('\033[1m'+'NO verifica calibracion')

Uc_ecal: 8.291297408218975e-05
delta_V: 8.765183431952661e-05
|ecal_m|+Uc_ecal = 8.294097408218975e-05
Verifica calibracion


##**Escala: 200mA**
##**Fuente: 6,75mA**

In [46]:
V_ver = [0.00675,0.00675,0.00676,0.00676,0.00675]
V_med = [6.8,6.8,6.7,6.8,6.8]#mA
V_ver_m = st.mean(V_ver)
V_med_m = st.mean(V_med)
ecal_m = V_med_m*mA - V_ver_m
print(f'V_ver_m: {V_ver_m}\nV_med_m: {V_med_m}\necal_m: {ecal_m}')

V_ver_m: 0.0067540000000000005
V_med_m: 6.78
ecal_m: 2.599999999999998e-05


###Incertidumbres de tipo A

In [47]:
V_med_mA = list(map(lambda x: x*mA, V_med))#paso de mA a Ampere

uA_V_ver = incertidumbre_A(V_ver)
uA_V_med = incertidumbre_A(V_med_mA)
print(f'Incertidumbre tipo A\nuA_V_ver: {uA_V_ver}\nuA_V_med: {uA_V_med}')

Incertidumbre tipo A
uA_V_ver: 2.4494897427832906e-06
uA_V_med: 1.999999999999988e-05


###Incertidumbres de tipo B

In [48]:
#valores sacados del datasheet del tester:
nd_ver = 6
er_p_ver = 0.17
Ci_V_ver = int(V_ver_m*100000) #son 6 digitos, y V_ver_m esta con 5 decimales

nd_med = 5
er_p_med = 1.2
Ci_V_med = int(V_med_m*10) #son 2 digitos, y V_med_m esta con 1 decimal

uB_V_ver = inecrtidumbre_B(er_p_ver, nd_ver, Ci_V_ver, V_ver_m)
uB_V_med = (res[2]/2)/np.sqrt(3)
print(f'Incertidumbre tipo B\nuB_V_ver: {uB_V_ver}\nuB_V_med: {uB_V_med}')

Incertidumbre tipo B
uB_V_ver: 4.1290564481730184e-05
uB_V_med: 2.8867513459481293e-05


###Incertidumbre total de cada magnitud e incertidumbre combinada - funcion error de calculo

In [49]:
uc_V_ver = uB_V_ver
uc_V_med = incertidumbre_total_A_B(uA_V_med, uB_V_med)
print(f'Incertidumbre total\nuc_V_ver: {uc_V_ver}\nuc_V_med: {uc_V_med}')

uc_ecal = mt.sqrt(uc_V_ver**2+uc_V_med**2)
print(f'uc_ecal: {uc_ecal}')

Incertidumbre total
uc_V_ver: 4.1290564481730184e-05
uc_V_med: 3.5118845842842394e-05
uc_ecal: 5.420557211720255e-05


###verificacion calibracion

In [50]:
k95 = 2
Uc_ecal = uc_ecal*k95
print(f'Uc_ecal: {Uc_ecal}')

delta_V = V_med_m*mA*((er_p_med/100)+(nd_med/Ci_V_med))
print(f'delta_V: {delta_V}')

suma = abs(ecal_m*mA)+ Uc_ecal
print(f'|ecal_m|+Uc_ecal = {suma}')

if suma <= delta_V: print('\033[1m' + 'Verifica calibracion')
else: print('\033[1m' + 'NO verifica calibracion')

Uc_ecal: 0.0001084111442344051
delta_V: 0.0005873301492537313
|ecal_m|+Uc_ecal = 0.0001084371442344051
Verifica calibracion
